# MPAR - Rapport du rendu II
**01/04/24**

## Introduction

Ce notebook présente l'explication, la démonstration et les résultats des outils développés dans le cadre du cours de MPAR. Les résultats de la première partie du rendu, discutés en cours, ne seront pas abordés ici mais seront uniquement utilisés comme référence.

## Objectifs

1. Comprendre les concepts de **Model Checking** et de **SMC (Statistical Model Checking)** pour les **chaînes de Markov**.
2. Étendre ces concepts aux **MDP (Markov Decision Process)**.
3. Utiliser le **RL (Reinforcement Learning)** pour améliorer le model checking statistique.

## Contenu
1. **Changements dans le parser**
2. **Chapitre 2, Vérification Probabiliste**
3. **Chapitre 3, Modélisation Probabiliste et Apprentissage par Renforcement**

## 1. Changements dans le parser

Pour pouvoir utiliser des modèles .mdp avec des recompenses, il a fallut modifier notre parser. Pour ceci, les anciens modèles .mdp, sans récompenses, doivent continuer a fonctionnner sans altération.

De cette façon, nous avons inclu la ligne optionnelle `Rewards` avec des entrées dans le format `s:r` avec s un état et r une récompense entière. L'exemple `states_with_rewards` montre ceci.

Ceci a été fait en ajoutant l'entrée (defrewards) optionnele (?)  a la définition du programme, ayant `program: defstates defrewards? defactions transitions EOF;` et ayant la définition `defewrards : REWARDS ID ':' INT (',' ID ':' INT)* ';';`.

Après, nous avons ajouté l'attribut `rewards` à la classe `gramPrintListener`, défini a travers les fonctions `enterDefrewards` et `update_rewards`.

De cette façon, comme dans l'exemple vu en cours, il suffit d'importer la fonction `run` de l'archive `mdp.py` pour lire un fichier et creer un objet de la classe `gramPrintListener` qui contiendrat, en plus des attributs vus dans la prémière démostration, les attributs de récompenses.

In [3]:
from mdp import run as run_mdp # Fonction qui crée l'objet a être lu.

Exemple sans récompenses:

In [4]:
simu_mc = run_mdp(path = "prof_examples//simu-mc.mdp", return_printer=True, print_transactions=True) # Exemple sans récompenses

ANTLR runtime and generated code versions disagree: 4.11.1!=4.13.1
ANTLR runtime and generated code versions disagree: 4.11.1!=4.13.1
Initialy declared states: ['I', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6']
Initialy declared actions: ['a']
Transition from I with no action and targets ['T1', 'T2'] with weights [1, 1]
Transition from T1 with no action and targets ['T3', 'T4'] with weights [1, 1]
Transition from T2 with no action and targets ['T5', 'T6'] with weights [1, 1]
Transition from T3 with no action and targets ['S1', 'T1'] with weights [1, 1]
Transition from T4 with no action and targets ['S2', 'S3'] with weights [1, 1]
Transition from T5 with no action and targets ['S4', 'S5'] with weights [1, 1]
Transition from T6 with no action and targets ['S6', 'T2'] with weights [1, 1]
Transition from S1 with no action and targets ['S1'] with weights [1]
Transition from S2 with no action and targets ['S2'] with weights [1]
Transition from S3 with no action and

Exemple avec récompenses:

In [5]:
states_with_rewards = run_mdp(path = "mdp_examples//states_with_rewards.mdp", return_printer=True, print_transactions=True) # Exemple avec récompenses

ANTLR runtime and generated code versions disagree: 4.11.1!=4.13.1
ANTLR runtime and generated code versions disagree: 4.11.1!=4.13.1
Initialy declared states: ['S0', 'S1', 'S2']
Initialy declared actions: ['a', 'b', 'c']
Transition from S0 with no action and targets ['S1', 'S2'] with weights [5, 5]
Transition from S1 with action b and targets ['S1', 'S0'] with weights [2, 8]
Transition from S1 with action a and targets ['S2', 'S0', 'S1', 'S3'] with weights [1, 3, 6, 2]
Transition from S2 with action c and targets ['S0', 'S1', 'S3'] with weights [5, 5, 10]
Transition from S2 with action d and targets ['S0', 'S3'] with weights [5, 7]
Transition from S3 with action e and targets ['S1', 'S2'] with weights [2, 2]

 ------- transactions df -------
  Origin Action   S0   S1   S2    S3
0     S0     NA  NaN    5    5   NaN
1     S1      b    8    2  NaN   NaN
2     S1      a    3    6    1   2.0
3     S2      c    5    5  NaN  10.0
4     S2      d    5  NaN  NaN   7.0
5     S3      e  NaN    2

Comme montré dans les exemples précedents, à chaque état nous avons un reward attribué. 

Si la récompense n'a pas été définie, notre parser attribut une valeur de zero. 

Il est un dictionnaire et peut être appelé facilement par l'attribut `rewards` comme suit:

In [6]:
print('states_with_reards.mdp', states_with_rewards.rewards)
print('simu_mc.mdp', simu_mc.rewards)

states_with_reards.mdp {'S0': 1, 'S1': 10, 'S2': 15, 'S3': 0}
simu_mc.mdp {'I': 0, 'T1': 0, 'T2': 0, 'T3': 0, 'T4': 0, 'T5': 0, 'T6': 0, 'S1': 0, 'S2': 0, 'S3': 0, 'S4': 0, 'S5': 0, 'S6': 0}


## 2. Vérification Probabiliste (Chapitre 2)

### Exercice 12: 
Identification des ensembles $S_0$, $S_1$ et $S_?$ pour une propriété $P(\diamond s)$ (fatalment $s$, où $s$ est un état).

#### Solution:

Nous avons décidé d'utiliser notre dataframe de probabilitées pour raisoner. L'idée geral de notre algorithme est:
1. Au début, aucun état ne peut arriver ($S_{0} = all states $). Nous ne deplaçons les états seulement depuis $S_0$ vers les autres ensembles, jamais le chemin inverse où de $S_?$ vers $S_1$ où vice versa.
2. L'état objectif satisfait toujours la propriété, alors on le déplace de $S_{0}$ pour $S_1$
3. Maitenant, nous voulons trouver tous les états $S_1$. Pour ça, il faut faire une boucle.
    - Nous calculons la probabilité totale de chaque couple (état, action) d'arriver aux états de $S_1$. 
    - Pour chaque état, entre toutes les actions possibles qu'il peut avoir, si la probabilité totale minimum est de 1, alors toutes sont égals a 1, donc il est déplacé pour $S_1$.
    - Si $S_1$ a changé, nous recommençons le calcul. 
4. Maitenant, pour trouver les états $S_?$, nous refaisons une boucle.
    - Idem pour la probabilité d'arrivé, mais cette fois-ci en $S_1$ où $S_?$.
    - Ici, nous voulons l'ajouter a $S_?$ si au moins l'une de ces actions peut, peut-être, l'emmener a $S_?$ où $S_?$. Donc, il faut que la probabilité maximum d'un état soit plus grande que zéro.
    - Si $S_?$ a changé, nous recommençons le calcul. 


#### Idée de l'algo

L'idée de l'algorithme vient de la façon comme nous analisons un graphe pour trouver les ensembles. D'abord, nous començons toujours par $S_1$ et puis après par $S_?$. Aussi, a chaue fois nous regardons le graphe comme une seule chose au lieu de s'imaginer en marchant sur le graphe.

In [86]:
def segment_suremaynever_states(printer, target_state):
    # Initialize sets for S_sure, S_may, and S_never
    s_sure = set()
    s_may = set()
    s_never = set(printer.declared_states)
    s_sure.add(target_state)
    s_never.remove(target_state)
    
    stop = False
    # First cycle to add s_sures
    while not stop:
        stop = True
        probs = printer.transactions_prob.loc[:, ['Origin', 'Action']+list(s_sure)]
        probs['P_sure'] = probs.loc[:, list(s_sure)].sum(axis=1)
        for o in probs['Origin'].unique():
            if probs.loc[probs['Origin']==o]['P_sure'].min() == 1: # It's sure if all actions lead to 100% prob of arriving to sure. 
                    if o in s_never:                               # Si un nouveau état est ajouté, on continue le cicle
                        s_never.remove(o)
                        s_sure.add(o)
                        stop = False
    stop = False
    # First cycle to add s_sures
    while not stop:
        stop = True
        probs = printer.transactions_prob.loc[:, ['Origin', 'Action']+list(s_may)+list(s_sure)]
        probs['P_may'] = probs.loc[:, list(s_sure)+list(s_may)].sum(axis=1)
        for o in probs['Origin'].unique():
            if probs.loc[probs['Origin']==o]['P_may'].max() > 0: # It may arrive if at least one action have a probability of arriving to a may or sure. 
                    if o in s_never:                             # Si un nouveau état est ajouté, on continue le cicle
                        s_never.remove(o)
                        s_may.add(o)
                        stop = False
    
    return list(s_sure), list(s_may), list(s_never)

In [87]:
smaysurenever = run_mdp(path = "mdp_examples//smaysurenever.mdp", return_printer=True, print_transactions=False) 

ANTLR runtime and generated code versions disagree: 4.11.1!=4.13.1
ANTLR runtime and generated code versions disagree: 4.11.1!=4.13.1
Initialy declared states: ['S0', 'S1', 'S2', 'S3']
Initialy declared actions: ['a', 'b', 'c']
Transition from S0 with action a and targets ['S1', 'S2'] with weights [1, 1]
Transition from S0 with action b and targets ['S0'] with weights [1]
Transition from S11 with no action and targets ['S2'] with weights [1, 1]
Transition from S2 with no action and targets ['S3'] with weights [1]
Transition from S1 with no action and targets ['S3'] with weights [1]
Transition from S5 with no action and targets ['S6'] with weights [1]

 ---------- WARNINGS WHEN PARSING -----------
( 0 ) - Undeclared state in transition: S11, declared automaticaly
( 1 ) - Undeclared state in transition: S5, declared automaticaly
( 2 ) - Undeclared state S6 targeted in transition from S5 with NA, declared automaticaly
( 3 ) - State S0 reward wasn't assigned, using zero as reward
( 4 ) - S

line 5:11 mismatched input ',' expecting {';', '+'}


In [89]:
S_sure, S_may, S_never = segment_suremaynever_states(smaysurenever, 'S3')
S_sure, sorted(S_may), sorted(S_never)

(['S2', 'S1', 'S11', 'S3'], ['S0'], ['S5', 'S6'])

### Exercice 13:

A partir d'un MDP $ M = (S, \text{Act}, P, \iota_{\text{init}}, AP, L) $ et une propriété $ P_{\text{max}} (\diamond s^*) $ avec $ s^* \in S $ un état donné. Proposer une définition pour la matrice $ A $ et le vecteur $ b $ du programme linéaire $ A \cdot x \geq b $. Calculer $x$.


#### Solution

Ici, nous voulons une solution gérale, qui marche pour des MDP mais aussi pour des MC. Pour ça, cest important de réetablire les vecteurs et matrices d'inéquations et équations a `None` quand nous n'avons pas d'équation où inéquation.

In [440]:
import numpy as np
from scipy.optimize import linprog

def solve_system(printer, S_may, S_sure):
    '''
    Fonctionne pour des MC et des MDP
    '''
    df = printer.transactions_prob
    # Initialize lists for inequality and equality constraints
    A_ub, b_ub, A_eq, b_eq = [], [], [], []
    Ubfollower = []
    # Iterate over source states in S_may
    for source_state in S_may:
        # Check if there is only one transition from the source state
        if len(df.loc[df['Origin'] == source_state]) == 1:
            # If only one transition, add it as an equality constraint
            t = df.loc[df['Origin'] == source_state, S_may].copy()
            t.loc[:, t.columns == source_state] -= 1  # Identity matrix substracted (A-I), but we invert later
            A_eq.append(-t.values)                    # (A-I) becomes (I-A)
            b_eq.append(np.sum(df.loc[df['Origin'] == source_state, S_sure].values, axis=1))
        else:
            # If multiple transitions, add them as inequality constraints
            mask_state = df['Origin'] == source_state
            Ubfollower.append(source_state)
            ts = df.loc[mask_state, S_may].copy()
            ts.loc[:, ts.columns == source_state] -=1
            Ubfollower.append(list(df.loc[mask_state, 'Action']))
            for i in range(len(df.loc[mask_state])):
                A_ub.append(ts.values[i])
                b_ub.append(-np.sum(df.loc[df['Origin'] == source_state, S_sure].values, axis=1)[i])
    
    c = np.ones(len(S_may)) # Objectif: Minimizer la somme des x de chaque état
        
    A_ub, b_ub, A_eq, b_eq = [None if not v else v for v in [A_ub, b_ub, A_eq, b_eq]]
    A_eq, A_ub = [np.vstack(m) if m is not None else None for m in [A_eq, A_ub]]

    # Solve the linear programming problem
    print('--------- System constraints:')
    print(f"{Ubfollower=}")
    print(f"{A_ub=}")
    print(f"{b_ub=}")
    print(f"{A_eq=}")
    print(f"{b_eq=}")
    print(f"{c=}")

    res = linprog(c=c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=(0,1))

    return res

In [441]:
def abc_analysis(printer, state):
    S_sure, S_may, S_never = segment_suremaynever_states(printer, state)
    S_sure, sorted(S_may), sorted(S_never)

    print('--------- S segments:')
    print(f'{S_sure=}')
    print(f'{S_may=}')
    print(f'{S_never=}')

    res = solve_system(printer, S_may, S_sure)

    print('--------- Solution:')
    print(f'{res.x=}')
    print(f'{res.fun=}')
    print(f'{res.success=}')
    print(f'{res.slack=}')
    print(f'{res.con=}')

#### Testant pour une MC:

In [443]:
mc_c2p41 = run_mdp(path = "mdp_examples//exemple_cours_c2p41_mc.mdp", return_printer=True, print_transactions=False) # Exemple sans récompenses

ANTLR runtime and generated code versions disagree: 4.11.1!=4.13.1
ANTLR runtime and generated code versions disagree: 4.11.1!=4.13.1
Initialy declared states: ['II', 'CI', 'AI', 'CC', 'CA', 'AC', 'AA', 'F', 'L']
Initialy declared actions: ['NA']
Transition from II with no action and targets ['CI', 'AI'] with weights [2, 1]
Transition from CI with no action and targets ['CC'] with weights [1]
Transition from CC with no action and targets ['F', 'L'] with weights [1, 1]
Transition from CA with no action and targets ['F'] with weights [1]
Transition from AC with no action and targets ['F'] with weights [1]
Transition from AA with no action and targets ['L'] with weights [1]
Transition from AI with no action and targets ['AA'] with weights [1]

 ---------- WARNINGS WHEN PARSING -----------
( 0 ) - State II reward wasn't assigned, using zero as reward
( 1 ) - State CI reward wasn't assigned, using zero as reward
( 2 ) - State AI reward wasn't assigned, using zero as reward
( 3 ) - State CC 

In [444]:
abc_analysis(mc_c2p41, 'F')

--------- S segments:
S_sure=['F', 'AC', 'CA']
S_may=['II', 'CI', 'CC']
S_never=['L', 'AI', 'AA']
--------- System constraints:
Ubfollower=[]
A_ub=None
b_ub=None
A_eq=array([[ 1.        , -0.66666667, -0.        ],
       [-0.        ,  1.        , -1.        ],
       [-0.        , -0.        ,  1.        ]])
b_eq=[array([0.]), array([0.]), array([0.5])]
c=array([1., 1., 1.])
--------- Solution:
res.x=array([0.33333333, 0.5       , 0.5       ])
res.fun=1.3333333333333333
res.success=True
res.slack=array([], dtype=float64)
res.con=array([0., 0., 0.])


#### Testing for a simple MDP

In [445]:
simple_mdp = run_mdp(path = "mdp_examples//simple_mdp.mdp", return_printer=True, print_transactions=False) # Exemple sans récompenses

ANTLR runtime and generated code versions disagree: 4.11.1!=4.13.1
ANTLR runtime and generated code versions disagree: 4.11.1!=4.13.1
Initialy declared states: ['I', 'W', 'L']
Initialy declared actions: ['a', 'b', 'c']
Transition from I with action a and targets ['W', 'L'] with weights [1, 1]
Transition from I with action b and targets ['W'] with weights [1]
Transition from I with action c and targets ['L'] with weights [1]
Transition from W with no action and targets ['W'] with weights [1]
Transition from L with no action and targets ['L'] with weights [1]

 ---------- WARNINGS WHEN PARSING -----------
( 0 ) - State I reward wasn't assigned, using zero as reward
( 1 ) - State W reward wasn't assigned, using zero as reward
( 2 ) - State L reward wasn't assigned, using zero as reward


In [446]:
abc_analysis(simple_mdp, 'W')

--------- S segments:
S_sure=['W']
S_may=['I']
S_never=['L']
--------- System constraints:
Ubfollower=['I', ['a', 'b', 'c']]
A_ub=array([[-1.],
       [-1.],
       [-1.]])
b_ub=[-0.5, -1.0, -0.0]
A_eq=None
b_eq=None
c=array([1.])
--------- Solution:
res.x=array([1.])
res.fun=1.0
res.success=True
res.slack=array([0.5, 0. , 1. ])
res.con=array([], dtype=float64)


#### Testing for another MDP

In [447]:
mdp_c2p40 = run_mdp(path = "mdp_examples//exemple_cours_c2p40_mdp.mdp", return_printer=True, print_transactions=False) # Exemple sans récompenses

ANTLR runtime and generated code versions disagree: 4.11.1!=4.13.1
ANTLR runtime and generated code versions disagree: 4.11.1!=4.13.1
Initialy declared states: ['II', 'CI', 'AI', 'CC', 'CA', 'AC', 'AA', 'F', 'L']
Initialy declared actions: ['NA', 'a', 'p', 'c']
Transition from II with action c and targets ['CI'] with weights [1]
Transition from II with action p and targets ['CI', 'AI'] with weights [2, 1]
Transition from II with action a and targets ['AI'] with weights [1]
Transition from CI with action a and targets ['CA'] with weights [1]
Transition from CI with action c and targets ['CC'] with weights [1]
Transition from CI with action p and targets ['CC', 'CA'] with weights [2, 1]
Transition from CC with no action and targets ['L', 'F'] with weights [1, 1]
Transition from CA with no action and targets ['F'] with weights [1]
Transition from F with no action and targets ['F'] with weights [1]
Transition from L with no action and targets ['L'] with weights [1]
Transition from AI with 

In [448]:
abc_analysis(mdp_c2p40, 'F')

--------- S segments:
S_sure=['F', 'AC', 'CA']
S_may=['II', 'AI', 'CI', 'CC']
S_never=['L', 'AA']
--------- System constraints:
Ubfollower=['II', ['c', 'p', 'a'], 'AI', ['c', 'a', 'p'], 'CI', ['a', 'c', 'p']]
A_ub=array([[-1.        ,  0.        ,  1.        ,  0.        ],
       [-1.        ,  0.33333333,  0.66666667,  0.        ],
       [-1.        ,  1.        ,  0.        ,  0.        ],
       [ 0.        , -1.        ,  0.        ,  0.        ],
       [ 0.        , -1.        ,  0.        ,  0.        ],
       [ 0.        , -1.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        , -1.        ,  0.        ],
       [ 0.        ,  0.        , -1.        ,  1.        ],
       [ 0.        ,  0.        , -1.        ,  0.66666667]])
b_ub=[-0.0, -0.0, -0.0, -1.0, -0.0, -0.6666666666666666, -1.0, -0.0, -0.3333333333333333]
A_eq=array([[-0., -0., -0.,  1.]])
b_eq=[array([0.5])]
c=array([1., 1., 1., 1.])
--------- Solution:
res.x=array([1. , 1. , 1. , 0.5])
res.fun=3.5

### Exercice 14

Expliquer comment adapter les algorithmes de vérification de l'exercice 13 pour le calcul de la récompense attendue pour des modèles de récompense Markoviens (MC et MDP).

### Exercice 15
Simulateur de lancers de pièces successifs pour simuler un dé à 6 faces

### Exercice 16
Algorithme de SMC quantitatif - simulant un dé à 6 faces

### Exercice 17.1

Implémenter l’algorithme de SMC qualitatif utilisant SPRT (Sequential Probability Ratio Test [Wald, 1945]) dans le programme de l’exercice 15.

### Exercice 17.2
On appelle $p_{ki}$ la probabilité d’obtenir le chiffre $i$ après $k$ lancers au maximum. Utiliser votre programme pour estimer $P(p_{ki} \geq 0.1)$, $P(p_{ki} \geq 0.14)$, $P(p_{ki} \geq 0.16)$. Qu’observez-vous ? Utiliser $\alpha = \beta = 0.01$.

### Exercice 17.3
Tester sur le modèle du CRAPS.

### Exercice 17.4
Tester sur un MDP

## 3. Modélisation Probabiliste et Apprentissage par Renforcement (Chapitre 3)